In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import datetime as dt
import itertools

In [2]:
data = pd.read_csv('movie_bd_v5.csv') # загружаем данные
data.sample(5)                        # просмотр 5 случайных образцов

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1197,tt1453405,200000000,743559607,Monsters University,Billy Crystal|John Goodman|Steve Buscemi|Helen...,Dan Scanlon,School never looked this scary.,A look at the relationship between Mike and Su...,104,Animation|Family,Walt Disney Pictures|Pixar Animation Studios,6/20/2013,7.0,2013
136,tt2872718,8500000,38697217,Nightcrawler,Jake Gyllenhaal|Rene Russo|Riz Ahmed|Bill Paxt...,Dan Gilroy,The city shines brightest at night,"When Lou Bloom, desperate for work, muscles in...",117,Crime|Drama|Thriller,Bold Films|Sierra / Affinity,10/23/2014,7.6,2014
173,tt1086772,40000000,123494610,Blended,Drew Barrymore|Adam Sandler|Wendi McLendon-Cov...,Frank Coraci,"Single Dad, No Clue. Single Mum, Flying Solo.","After a bad blind date, a man and woman find t...",117,Comedy,Happy Madison Productions|Karz Entertainment|W...,5/21/2014,6.7,2014
319,tt1186367,40000000,60462347,Ninja Assassin,Rain|Naomie Harris|Sung Kang|Randall Duk Kim|R...,James McTeigue,"Fear not the weapon, but the hand that wields it.","Ninja Assassin follows Raizo (Rain), one of th...",99,Action|Crime|Thriller,Studio Babelsberg|Anarchos Productions|Legenda...,9/29/2009,6.1,2009
1470,tt0385880,75000000,140175006,Monster House,Ryan Newman|Steve Buscemi|Mitchel Musso|Sam Le...,Gil Kenan,The House is . . . ALIVE!,"Monsters under the bed are scary enough, but w...",91,Animation|Comedy|Family|Fantasy,Columbia Pictures|Amblin Entertainment|Sony Pi...,7/21/2006,6.3,2006


# Предобработка

In [3]:
answers = {} # создадим словарь для ответов

data['profit'] = data.revenue - data.budget # создадим новый столбец - прибыль

def my_sort(x):
    '''Функция возвращает отсортированный список'''
    x.sort()
    return x

In [4]:
data.describe() # очень полезно знать характеристики полей

,budget,revenue,runtime,vote_average,release_year,profit
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000,1.889000e+03
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773,1.010545e+08
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841,1.830733e+08
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000,-1.657101e+08
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000,5.000000e+06
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000,4.400041e+07
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000,1.201842e+08
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000,2.544506e+09


In [5]:
data.info() # также полезно знать формат данных DataFrame

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
 14  profit                1889 non-null   int64  
dtypes: float64(1), int64(

In [6]:
data.sample(5) # просмотр 5 случайных образцов обновлённого DataFrame

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
1875,tt0219854,65000000,69700000,The Kid,Bruce Willis|Spencer Breslin|Emily Mortimer|Li...,Jon Turteltaub,Nobody ever grows up quite like they imagined.,Powerful businessman Russ Duritz is self-absor...,104,Fantasy|Comedy|Family,Walt Disney Pictures,7/7/2000,5.9,2000,4700000
819,tt1033575,20000000,177243185,The Descendants,George Clooney|Shailene Woodley|Amara Miller|N...,Alexander Payne,The South Pacific ain't that terrific.,With his wife Elizabeth on life support after ...,115,Comedy|Drama,Fox Searchlight Pictures|Dune Entertainment|Ad...,9/9/2011,6.6,2011,157243185
407,tt0840361,37000000,154026136,The Town,Ben Affleck|Rebecca Hall|Jeremy Renner|Jon Ham...,Ben Affleck,Welcome to the bank robbery capital of America.,"Doug MacRay is a longtime thief, who, smarter ...",125,Crime|Drama|Thriller,Legendary Pictures|GK Films|Thunder Road Pictu...,9/15/2010,6.9,2010,117026136
729,tt0945513,32000000,147332697,Source Code,Jake Gyllenhaal|Michelle Monaghan|Vera Farmiga...,Duncan Jones,Make Every Second Count,Decorated soldier Captain Colter Stevens wakes...,93,Thriller|Science Fiction|Mystery,The Mark Gordon Company|Vendome Pictures,3/30/2011,7.0,2011,115332697
1363,tt0452598,60000000,129181830,Cheaper by the Dozen 2,Steve Martin|Eugene Levy|Bonnie Hunt|Tom Welli...,Adam Shankman,Same Big Family... Even Bigger Adventure.,Steve Martin and Bonnie Hunt return as heads o...,94,Adventure|Comedy|Family,Twentieth Century Fox Film Corporation,12/21/2005,5.8,2005,69181830


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

*Но если я получил их честно, то считаю, что я могу*
*заполнить словарь правильным ответом :)*

In [7]:
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# +
# в первом варианте использовалось стандартные подходы фильтрации:
#     сравнение поля budget с максимальном значением по столбцу
# во втором варианте использовалась полезная функция idxmax()

# P.S. не смотрел..

ВАРИАНТ 1

In [8]:
data[data.budget == data.budget.max()].original_title.iloc[0]

'Pirates of the Caribbean: On Stranger Tides'

ВАРИАНТ 2

In [9]:
data.loc[data.budget.idxmax()].original_title

'Pirates of the Caribbean: On Stranger Tides'

# 2. Какой из фильмов самый длительный (в минутах)?

In [10]:
answers['2'] = 'Gods and Generals (tt0279111)'
# +
# по аналогии в первом варианте использовалось
# стандартные подходы фильтрации:
#     сравнение поля budget с максимальном значением по столбцу
# а во втором варианте использовалась полезная функция idxmax()

# P.S. аналогично, не смотрел..

ВАРИАНТ 1

In [11]:
data[data.runtime == data.runtime.max()].original_title.iloc[0]

'Gods and Generals'

ВАРИАНТ 2

In [12]:
data.loc[data.runtime.idxmax()].original_title

'Gods and Generals'

# 3. Какой из фильмов самый короткий (в минутах)?





In [13]:
answers['3'] = 'Winnie the Pooh (tt1449283)'
# +
# аналогично задачам 1, 2

# P.S. тоже не смотрел этот мультфильм

ВАРИАНТ 1

In [14]:
data[data.runtime == data.runtime.min()].original_title.iloc[0]

'Winnie the Pooh'

ВАРИАНТ 2

In [15]:
data.loc[data.runtime.idxmin()].original_title

'Winnie the Pooh'

# 4. Какова средняя длительность фильмов?


In [16]:
answers['4'] = 110
# +
# для решения задачи используется агрегирующая функция mean

# P.S. по ощущениям так и есть.

In [17]:
round(data.runtime.mean()) # для округления используем round

110

# 5. Каково медианное значение длительности фильмов? 

In [18]:
answers['5'] = 107
# +
# для решения задачи используется агрегирующая функция median
# round использовался для получения целого типа

# P.S. слабо отличается от средненего..

In [19]:
round(data.runtime.median()) # для округления используем round

107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [20]:
answers['6'] = 'Avatar (tt0499549)'
# +
# решается по аналогии с предыдущими задачами

# P.S. смотрел предвзято, много позже после выхода
#      сюжет избит, спецэффекты на высоте, но
#      не цепляет, как в детстве Т800;
#      и склонен думать, что ГГ - предатель :)

ВАРИАНТ 1

In [21]:
data[data.profit == data.profit.max()].original_title.iloc[0]

'Avatar'

ВАРИАНТ 2

In [22]:
data.loc[data.profit.idxmax()].original_title

'Avatar'

# 7. Какой фильм самый убыточный? 

In [23]:
answers['7'] = 'The Lone Ranger (tt1210819)'
# +
# решается по аналогии с предыдущими задачами

# P.S. не смотрел и этот фильм..

ВАРИАНТ 1

In [24]:
data[data.profit == data.profit.min()].original_title.iloc[0]

'The Lone Ranger'

ВАРИАНТ 2

In [25]:
data.loc[data.profit.idxmin()].original_title

'The Lone Ranger'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [26]:
answers['8'] = 1478
# +
# вариант 1 путём фильтрации, а затем вызывается
# агрегирующая функия count для одного из столбцов
# вариант 2 некое велосипедостроение, скорее показывает,
# что штатные функции Python-a всё ещё strong

# P.S. это ~80%

ВАРИАНТ 1

In [27]:
data[data.revenue > data.budget]['imdb_id'].count()

1478

ВАРИАНТ 2

In [28]:
len(data[data.revenue > data.budget])

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [29]:
answers['9'] = 'The Dark Knight (tt0468569)'
# +
# первый вариант использует сортировку по столбцу revenue
# во втором варианте использовался старый подход (1, 2,..)
# P.S. фильм смотрел, показался нудным; как сказал Д. Пучков
# всю соль убили переводчики. И чтоб насладиться им, надо
# смотреть в оригинале, но на слух я вообще не понимаю :(

ВАРИАНТ 1

In [30]:
data.loc[data.release_year.isin([2008])].sort_values(by='revenue', ascending=False)['original_title'].iloc[0]

'The Dark Knight'

In [31]:
data_2008 = data[data.release_date.str.contains('2008')].copy() # копируем отфильтрованный DataFrame
data_2008.loc[data_2008.revenue.idxmax()].original_title        # дальше по аналогии с задачами 1, 2,..

'The Dark Knight'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [32]:
answers['10'] = 'The Lone Ranger'
# +
# решается по аналогии с предыдущими задачей 9
# можно было отфильтровать, как в 1, 2,..

# P.S. понятия не имею о чём он..

In [33]:
data.loc[data.release_year.isin([2012, 2013, 2014])].sort_values(by='profit')['original_title'].iloc[0]

'The Lone Ranger'

# 11. Какого жанра фильмов больше всего?

In [34]:
answers['11'] = 'Drama'
# +
# Два подхода, один стандартный, второй со сводной таблицей
# Дополнительно модифицируем столбец "genres" функцией str.split

# P.S. никогда не задумывался об этом

ВАРИАНТ 1

In [35]:
data.genres.str.split('|').explode().value_counts().index[0] # В одну строчку

'Drama'

ВАРИАНТ 2

In [36]:
# Создаём копию DataFrame
genres_data = data.copy()

# Обновляем поле "genres", раскрывая его посредством функции explode
genres_data.genres = data.genres.str.split('|').explode('genres')

# Создаём сводную таблицу: в строках жанры, в столбцах идентификаторы фильмов, в качестве значений название фильмов
genres_pivot = genres_data.pivot_table(index = 'genres', columns = 'imdb_id', values = 'original_title', aggfunc = 'count', fill_value = 0)

# Выбираем максимальную сумму по строкам
genres_pivot.sum(axis=1).idxmax()

'Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [37]:
answers['12'] = 'Drama'
# +
# Уже стандартные подходы, но во втором вспомнил, что есть столбец "profit"

# P.S. Ok. Буду знать.

ВАРИАНТ 1

In [38]:
data[data.revenue > data.budget].genres.str.split('|').explode().value_counts().index[0]

'Drama'

ВАРИАНТ 2

In [39]:
data[data.profit > 0].genres.str.split('|').explode().value_counts().index[0]

'Drama'

ВАРИАНТ 3

In [40]:
# Создаём DataFrame bb_data (сокр. blockbuster_data)
bb_data = data.query('profit > 0').copy()

# Преобразуем bb_data в Series, используя функции, split и explode
bb_data = bb_data.genres.str.split('|').explode()

# Создаём сумму по возрастанию и выводим первый элемент
bb_data.value_counts().index[0]

'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [41]:
answers['13'] = 'Peter Jackson'
# +
# В первом варианте группируем по "director", а дальше выполняем стандартные
# относительно столбца "revenue"
# Во втором случае очень полезной оказалась сводная таблица и параметр axis
# позволивший сделать суммирование по строкам

# P.S. Чего только Властилин Колец стоит. В переводе Д.Пучкова веселей :)

ВАРИАНТ 1

In [42]:
data.groupby('director').revenue.sum().sort_values(ascending=False).index[0]

'Peter Jackson'

ВАРИАНТ 2

In [43]:
# Создаём сводную таблицу: в строках ФИО, в столбцах идентификаторы фильмов, в качестве значений сборы
director_pivot = data.pivot_table(index='director', columns='imdb_id', values='revenue', aggfunc='sum', fill_value=0)

# Выбираем максимальную сумму по строкам
director_pivot.sum(axis=1).idxmax()

'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [44]:
answers['14'] = 'Robert Rodriguez'
# +
# Все 3 варианта дают идентичные ответы..
# Даже если непосредственно отфильтровать по Бэю, то у него 7 картин
# Но в силу того, что ни Бэя, ни Андерсона нет в вариантах ответа, то
# выбираем какого-то Родригеса.

# P.S. Согласен с комментарием в поле ответов - цифры говорят другое.

ВАРИАНТ 1

In [45]:
data[data.genres.str.contains('Action')].director.value_counts()

Paul W.S. Anderson    7
Michael Bay           7
Ridley Scott          6
Robert Rodriguez      6
Antoine Fuqua         6
                     ..
Frank Miller          1
Wayne Kramer          1
Ilya Naishuller       1
David Barrett         1
Simon Hunter          1
Name: director, Length: 345, dtype: int64

ВАРИАНТ 2

In [46]:
# Создаём новый DataFrame
action_data = data[data.genres.str.contains('Action')].copy()

# Создаём сводную таблицу: в строках ФИО, в столбцах идентификаторы фильмов, в качестве значений сборы
action_pivot = action_data.pivot_table(index='director', columns='imdb_id', values='genres', aggfunc='count', fill_value=0)
#display(action_pivot)

# Выбираем максимальную сумму по строкам
action_pivot.sum(axis=1).sort_values(ascending=False)

director
Michael Bay           7
Paul W.S. Anderson    7
Robert Rodriguez      6
Ridley Scott          6
Antoine Fuqua         6
                     ..
Les Mayfield          1
Kurt Wimmer           1
Kevin Smith           1
Kevin Macdonald       1
Adam McKay            1
Length: 345, dtype: int64

ВАРИАНТ 3

In [47]:
data[data.genres.str.contains('Action')].groupby(by='director').imdb_id.count().sort_values(ascending=False)

director
Michael Bay           7
Paul W.S. Anderson    7
Robert Rodriguez      6
Ridley Scott          6
Antoine Fuqua         6
                     ..
Les Mayfield          1
Kurt Wimmer           1
Kevin Smith           1
Kevin Macdonald       1
Adam McKay            1
Name: imdb_id, Length: 345, dtype: int64

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [48]:
answers['15'] = 'Chris Hemsworth'
# +
# Первый вариант стандартным путём решался
# Второй вариант с помощью сводной таблицы

# P.S. Кто эти люди?..

ВАРИАНТ 1

In [49]:
# Копируем DataFrame
actor_data = data.copy()

# Фильтруем
actor_data = actor_data[actor_data.release_year == 2012]

# Обновляем поле "cast", фильтруя и раскрывая его посредством функции explode
actor_data.cast = actor_data.cast.str.split('|')
actor_data = actor_data.explode('cast')

# Группируем по актёрам, суммируем по сборам и сортируем
actor_data.groupby('cast').revenue.sum().sort_values(ascending=False).index[0]

'Chris Hemsworth'

ВАРИАНТ 2

In [50]:
# Создаём копию исходного DataFrame
best_actor = data.copy()

# Обновляем поле "cast", фильтруя и раскрывая его посредством функции explode
best_actor.cast = data[data.release_year == 2012].cast.str.split('|')
best_actor = best_actor.explode('cast')

# Создаём сводную таблицу в качестве строк ФИО, в качестве столбцов идентификатор, в качестве значений сборы
best_pivot = best_actor.pivot_table(index='cast', columns='imdb_id', values='revenue', fill_value=0)
best_pivot.sum(axis=1).sort_values(ascending=False).index[0]

'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [51]:
answers['16'] = 'Matt Damon'
# +
# Использовался стандартный подход: фильтрация, модификация, эксплоуд..

# P.S. Особенно понравился в Отступниках, ну и картошка на Марсе. Борн меньше)

In [52]:
data[data.budget>data.budget.mean()]['cast'].str.split('|').explode().value_counts().index[0]

'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [53]:
answers['17'] = 'Action'
# +
# Использовался стандартный подход: фильтрация, модификация, эксплоуд..

# P.S. Я не удивлён.

In [54]:
data[data.cast.str.contains('Nicolas Cage')]['genres'].str.split('|').explode().value_counts().index[0]

'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [55]:
answers['18'] = 'K-19: The Widowmaker'
# +
# Фильтрация, сортировка, ..
# Люблю, когда в одну строчку

# P.S. В целом достойно наших показали, но нудно.

In [56]:
data[data.production_companies.str.contains('Paramount Pictures')].sort_values(by='profit')['original_title'].iloc[0]

'K-19: The Widowmaker'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [57]:
answers['19'] = 2015
# +
# Группировка, суммирование по сборам, сортировка

# P.S. Спорить не буду

In [58]:
data.groupby(by='release_year').revenue.sum().sort_values(ascending=False).index[0]

2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [59]:
answers['20'] = 2014
# +
# Фильтрация, группировка, суммирование по сборам, сортировка

# P.S. Я не против

In [60]:
data[data.production_companies.str.contains('Warner Bros')].groupby(by='release_year').profit.sum().sort_values(ascending=False).index[0]

2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [61]:
answers['21'] = 'September'
# +
# Используется преобразование для даты, затем выделяется месяц и value_counts()
# Испльзую последовательно lambda функцию

In [62]:
data.release_date.apply(pd.to_datetime).apply(lambda x: x.strftime('%B')).value_counts().index[0]

'September'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [63]:
answers['22'] = 450
# +
# Фильтрация, дальше стандартная функция nunique
# Также использовалась полезная lambda функция

# P.S. Почти четверть датасета. Полагаю всё поровну для всех времён года

In [64]:
data[data.release_date.apply(pd.to_datetime).apply(lambda x: x.strftime('%m') in ['06', '07', '08'])].imdb_id.nunique()

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [65]:
answers['23'] = 'Peter Jackson'
# +
# В первом варианте использовался стандартный подход: фильтрация и value_counts
# Во втором и третьем - разные вариации на тему lambda функции

# P.S. Спорный вопрос, он Юж. Зеландии, там зимой - лето :)

ВАРИАНТ 1

In [66]:
data[data.release_date.apply(pd.to_datetime).dt.month.isin([1, 2, 12,])].director.value_counts().index[0]

'Peter Jackson'

ВАРИАНТ 2

In [67]:
data[data.release_date.apply(pd.to_datetime).apply(lambda x: x.strftime('%m') in ['01', '02', '12'])].director.value_counts().index[0]

'Peter Jackson'

ВАРИАНТ 3

In [68]:
data[data.release_date.apply(pd.to_datetime).apply(lambda x: x.strftime('%B') in ["December", "January", "February"])].director.value_counts().index[0]

'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [69]:
answers['24'] = 'Four By Two Productions'
# +
# Оставляем списко как есть, так как может быть несколько компаний, а в ответе одна

# P.S. Что они хотя бы сняли?

In [70]:
# Копируем DataFrame
studio_data= data.copy()

# Модифицируем содержимое "original_title"
studio_data.original_title = studio_data.original_title.str.len()

# Разбиваем и раскрываем по "production_companies"
studio_data.production_companies = studio_data.production_companies.str.split('|')
studio_data = studio_data.explode('production_companies')

# Группируем по компаниям, выбираем максимумы и сортируем
studio_data.groupby(by='production_companies').max().original_title.sort_values(ascending=False)

production_companies
Four By Two Productions                   83
Twentieth Century Fox Film Corporation    83
Walden Media                              62
Walt Disney                               62
Jim Henson Company, The                   59
                                          ..
Everest Entertainment                      3
Berlanti Productions                       3
XM2 Productions                            2
Global Entertainment Group                 2
Ixtlan Productions                         2
Name: original_title, Length: 1771, dtype: int64

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [71]:
answers['25'] = 'Midnight Picture Show'
# +
# Использовался стандартный подход с lambda функциями в первом варианте
# Во втором - сводная таблица
# Поскольку overview_data больше нигде не используется, решил оставить его в данной ячейке

# P.S. Даже не слышал о такой

ВАРИАНТ 1

In [72]:
# Копируем DataFrame
word_data = data.copy()

# Модифицируем содержимое "overview"
word_data.overview = word_data.overview.apply(lambda x: len(x.split()))

# Разбиваем и раскрываем по "production_companies"
word_data.production_companies = word_data.production_companies.str.split('|')
word_data = word_data.explode('production_companies')

# Группируем по компаниям вычисляем среднее и выводим нужную компанию
word_data.groupby(['production_companies'])['overview'].mean().idxmax()

'Midnight Picture Show'

ВАРИАНТ 2

In [73]:
# Копируем DataFrame
overview_data=data.copy()

# Создаём столбец "overview_len"
overview_data['overview_len'] = overview_data.overview.apply(lambda x:len(x.split(' ')))

# Разбиваем и раскрываем по "production_companies"
overview_data['production_companies'] = overview_data.production_companies.str.split('|')
overview_data = overview_data.explode('production_companies')

# Создаём сводную таблицу и сортируем по "overview_len"
overview_data.pivot_table(['overview_len'],['production_companies'], aggfunc='mean').sort_values(by='overview_len', ascending=False).index[0]

'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [74]:
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave'
# +
# Фильтрация, далее агрегирующая функция quantile
# Выбор ответа был обусловлен предложенными вариантами и полученной выборкой

# P.S. Будет время посмотрю, может и Дарк Найта с субтитрами огорю

In [75]:
data[data['vote_average'] > data['vote_average'].quantile(q=0.99)]['original_title']

9                                          Inside Out
34                                               Room
118                                      Interstellar
119                           Guardians of the Galaxy
125                                The Imitation Game
128                                         Gone Girl
138                          The Grand Budapest Hotel
370                                         Inception
599                                   The Dark Knight
872                                       The Pianist
1081    The Lord of the Rings: The Return of the King
1183                          The Wolf of Wall Street
1191                                 12 Years a Slave
1800                                          Memento
Name: original_title, dtype: object

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [76]:
answers['27'] = 'Daniel Radcliffe & Rupert Grint'
# +
# Используется библиотека itertools
# Ключевой момент для решения задачи - предварительная сортировка по актёрам,
# чтобы избежать получения ситуации, когда будут встречаться записи вроде
# "Л. Гурченко, Л. Куравлёв" и "Л. Куравлёв, Л. Гурченко"
# Вариант 1 предполагает модификацию поля, соответствующего актёрам, далее
# сортировка актёров, затем создание пар, затем эксплоуд и подсчёт
# В данном случае проведён трюк, где lambda функция неявно выполняет 2 действия:
# lambda x: [x.sort(), list(itertools.combinations(x, 2))][1] - сначала сортирует,
# а затем создаёт комбинации. На выход выдаёт элемент с индексом 1 - то что нужно!
# Второй подход ничем не отличается от предыдущего, но в нём я решил использовать
# хотя бы одну личную функцию на весь объём заданий.
# Выбор ответа был обусловлен предложенными вариантами и полученной выборкой

# P.S. Если не делать сортировку, то третье место займёт пара Rupert Grint+Emma Watson,
#      которые снялись вместе в 7 фильмах о Гарри Поттере. Но мы то знаем ;)

ВАРИАНТ 1

In [77]:
data.cast.str.split('|').apply(lambda x: [x.sort(), list(itertools.combinations(x, 2))][1]).explode().value_counts()

(Emma Watson, Rupert Grint)            8
(Daniel Radcliffe, Rupert Grint)       8
(Daniel Radcliffe, Emma Watson)        8
(Ben Stiller, Owen Wilson)             6
(Helena Bonham Carter, Johnny Depp)    6
                                      ..
(Devon Aoki, Rosario Dawson)           1
(Eddie Murphy, Rosario Dawson)         1
(Bobb'e J. Thompson, Paul Rudd)        1
(Ty Burrell, Ving Rhames)              1
(Jayma Mays, Jennifer Coolidge)        1
Name: cast, Length: 17943, dtype: int64

ВАРИАНТ 2

In [78]:
data.cast.str.split('|').apply(my_sort).apply(lambda x: list(itertools.combinations(x, 2))).explode().value_counts()

(Emma Watson, Rupert Grint)            8
(Daniel Radcliffe, Rupert Grint)       8
(Daniel Radcliffe, Emma Watson)        8
(Ben Stiller, Owen Wilson)             6
(Helena Bonham Carter, Johnny Depp)    6
                                      ..
(Devon Aoki, Rosario Dawson)           1
(Eddie Murphy, Rosario Dawson)         1
(Bobb'e J. Thompson, Paul Rudd)        1
(Ty Burrell, Ving Rhames)              1
(Jayma Mays, Jennifer Coolidge)        1
Name: cast, Length: 17943, dtype: int64

# Submission

In [79]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker',
 '19': 2015,
 '20': 2014,
 '21': 'September',
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe & Rupert Grint'}

In [80]:
# и убедиться что ни чего не пропустил)
len(answers)

27

## В целом считаю, что решил все задания используя стандартные возможности Python&Pandas. Одна функция не в счёт, так я внёс её умышленно, чтоб была.